In [12]:
import json
import subprocess

from collections import defaultdict
from sys import getsizeof

### New API

API key is required

In [13]:
RELEASE = 'latest'
DATASET = 'citations'
API_KEY = 'NpToKVNU472E8fiIPEu6N1KLG1ECGEju6xZe3wgN'

## API -> Direct Co-Citations

In [14]:
import networkx as nx
import pandas as pd
import requests

In [33]:
PAPER_URL = 'https://api.semanticscholar.org/graph/v1/paper/{0}?fields={1}'
PAPER_CITATION_FIELDS = 'citationCount,citations.paperId,citations.citationCount,' + \
    'citations.title,citations.url,citations.authors,citations.journal,citations.year'
CITATIONS_URL = 'https://api.semanticscholar.org/graph/v1/paper/{0}/citations?fields={1}&offset={2}&limit={3}'
CITATIONS_FIELDS = 'paperId,url,title,citationCount,authors,journal,year'
MAX_LIMIT = 1000
MAX_TOTAL_COUNT = 10000

In [84]:
groups = [
    # [
    #     '10.1016/j.neuroimage.2011.01.057'
    # ],
    [
        # '10.1038/nn.3101',
        '10.1016/j.neuroimage.2012.03.048'
    ],
    [
        # '10.1002/hbm.20346',
        # '10.1016/j.clinph.2004.04.029',
        # '10.1016/j.neuroimage.2011.11.084',
        # '10.1016/j.neuroimage.2011.01.055',
        # '10.1002/(SICI)1097-0193(1999)8:4<194::AID-HBM4>3.0.CO;2-C',
        '10.1103/PhysRevLett.100.234101',
        # '10.1098/rsta.2011.0081'
    ]
]

In [17]:
def get_paper_data(doi, fields):
    return requests.get(PAPER_URL.format(doi, fields)).json()

In [18]:
def get_citation_data(doi, fields, offset, limit=MAX_LIMIT):
    if offset + limit >= MAX_TOTAL_COUNT:
        limit = MAX_TOTAL_COUNT - offset - 1
    return requests.get(CITATIONS_URL.format(doi, fields, offset, limit)).json()

In [43]:
def authors_compact(authors):
    if not authors:
        return ''
    if len(authors) <= 3:
        return ', '.join([a['name'] for a in authors])
    else:
        return f"{authors[0]['name']} et al."

In [52]:
def journal_compact(journal):
    if not journal or 'name' not in journal:
        return ''
    result = journal['name']
    if 'volume' in journal:
        result += f" {journal['volume']}"
    if 'pages' in journal:
        result += f":{journal['pages']}"
    return result

In [50]:
def build_graph(groups):
    graph = nx.DiGraph()
    node_groups = []
    for group_papers in groups:
        nodes = []
        for paper_doi in group_papers:
            # Get paper data
            paper_data = get_paper_data(paper_doi, PAPER_CITATION_FIELDS)
            paper_id = paper_data.get('paperId', None)
            if not paper_id:
                print(f'Failed to find paper: {paper_doi}')
                continue

            citations = paper_data.get('citations', [])
            num_cit_retrieved = len(citations)
            num_cit_total = paper_data['citationCount']
            print(paper_doi)
            print(f'Retrieved {num_cit_retrieved} / {num_cit_total} citations')
            while num_cit_retrieved < num_cit_total:
                new_citations = get_citation_data(paper_doi, CITATIONS_FIELDS, num_cit_retrieved)
                citations.extend([cit['citingPaper'] for cit in new_citations.get('data', [])])
                num_cit_retrieved = new_citations.get('next', None)
                if num_cit_retrieved:
                    assert len(citations) == num_cit_retrieved, paper_doi
                else:
                    assert len(citations) == num_cit_total, paper_doi
                    num_cit_retrieved = num_cit_total
                print(f'Retrieved {num_cit_retrieved} / {num_cit_total} citations')

            # Add reversed edges to the graph for BFS
            citation_nodes = [(cit['paperId'], 
                              dict(citationCount=cit['citationCount'],
                                   url=cit['url'], title=cit['title'],
                                   year=cit.get('year', ''), 
                                   authors=authors_compact(cit.get('authors', [])),
                                   journal=journal_compact(cit.get('journal', {}))))
                              for cit in citations
                              if cit['paperId']]
            citation_edges = [(paper_id, cit['paperId']) 
                              for cit in citations
                              if cit['paperId']]  # skip None?

            # Add the node and incoming edges to the graph
            nodes.append(paper_id)
            graph.add_node(paper_id, citationCount=num_cit_total)            
            graph.add_nodes_from(citation_nodes)
            graph.add_edges_from(citation_edges)

        node_groups.append(nodes)

    return graph, node_groups

In [85]:
graph, node_groups = build_graph(groups)

10.1016/j.neuroimage.2012.03.048
Retrieved 270 / 270 citations
10.1103/PhysRevLett.100.234101
Retrieved 483 / 483 citations


In [86]:
print(graph.number_of_nodes())
print(graph.number_of_edges())
print(node_groups)

725
750
[['9298bc29443bdfe48bed12456fabd3e8e7af03ed'], ['d63e6d45be6f44c2078eb7266f8a73a31fb72565']]


In [87]:
crosschecked = set(graph.nodes)
for group_nodes in node_groups:
    reachable = nx.bfs_layers(graph, group_nodes)
    print(next(reachable))  # skip group nodes in layer 0
    crosschecked &= set(next(reachable))

['9298bc29443bdfe48bed12456fabd3e8e7af03ed']
['d63e6d45be6f44c2078eb7266f8a73a31fb72565']


In [88]:
len(crosschecked)

26

In [89]:
crosschecked_data = [dict(paperId=n[0], **n[1]) for n in graph.nodes(data=True) if n[0] in crosschecked]

In [90]:
crosschecked_df = pd.DataFrame(crosschecked_data)

In [91]:
crosschecked_df.sort_values(by='citationCount', ascending=False).head(n=20)

,paperId,citationCount,url,title,year,authors,journal
20,fa26a5432c5cd27f6c7469d61496301db6adae42,280,https://www.semanticscholar.org/paper/fa26a543...,Opportunities and methodological challenges in...,,,
16,792fec9ba1be6b80dec5bd78c0f954583621856d,277,https://www.semanticscholar.org/paper/792fec9b...,How reliable are MEG resting-state connectivit...,,,
13,967792df32a9abc2f83971c0a43d821b29944f27,146,https://www.semanticscholar.org/paper/967792df...,Consistency of EEG source localization and con...,,,
12,057cc18f57cd9ebebf866791d8600538ff7e1961,131,https://www.semanticscholar.org/paper/057cc18f...,Dynamics of large-scale electrophysiological n...,,,
19,10cb59e29c26dabd9e24edf09d367bedb65f884a,88,https://www.semanticscholar.org/paper/10cb59e2...,Measuring electrophysiological connectivity by...,,,
10,c912e7d61fb7345934b15e256bfe47226dce7837,85,https://www.semanticscholar.org/paper/c912e7d6...,IFCN-endorsed practical guidelines for clinica...,,,
24,e2c4a75f1f4b7d0f84c172f9702841b7e6d805b9,75,https://www.semanticscholar.org/paper/e2c4a75f...,"Cognitive and Clinical Dysfunction, Altered ME...",,,
21,39fa92f61b329e15c6efe0c06891b8ded408512e,73,https://www.semanticscholar.org/paper/39fa92f6...,Dynamic recruitment of resting state sub-networks,,,
25,5d1c6e9052f8cd4f50847bb456236b80c3f9c852,61,https://www.semanticscholar.org/paper/5d1c6e90...,Task induced modulation of neural oscillations...,,,
23,888e053190387c77e42a73191911e3c9cfce5886,49,https://www.semanticscholar.org/paper/888e0531...,Third order spectral analysis robust to mixing...,,,


### Old Way

In [76]:
DOWNLOAD_URL = 'https://s3-us-west-2.amazonaws.com/ai2-s2-research-public/open-corpus/2022-05-01/s2-corpus-{0:03d}.gz'
FILE_NAME_GZ = 's2-corpus-{0:03d}.gz'
FILE_NAME = 's2-corpus-{0:03d}'

In [79]:
def to_numeric(sha1_id):
    if sha1_id not in paper_ids:
        new_id = len(paper_ids)
        paper_ids[sha1_id] = new_id
        paper_index[new_id] = sha1_id
    return paper_ids[sha1_id]

In [83]:
def update_citations(corpus_id):
    p = subprocess.Popen(['wget', DOWNLOAD_URL.format(corpus_id)], 
                         stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    p.communicate()
    p.wait()
    
    p = subprocess.Popen(['gzip', '-d', FILE_NAME_GZ.format(corpus_id)], 
                         stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    p.communicate()
    p.wait()
    
    filename = FILE_NAME.format(corpus_id)
    with open(filename, 'r') as f:
        for line in f:
            data = json.loads(line)
            paper_id = to_numeric(data['id'])
            for cit_id in data['inCitations']:
                citations[to_numeric(cit_id)].add(paper_id)
            citations[paper_id].update(set(to_numeric(cit_id) for cit_id in data['outCitations']))
            
    p = subprocess.Popen(['rm', filename], 
                         stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    p.communicate()
    p.wait()

In [86]:
citations = defaultdict(set)
paper_ids = {}
paper_index = {}

print(f'Corpus ID\tNumPapers\tSizePapers\tNumCitations\tSizeCitations')
for corpus_id in range(10):
    update_citations(corpus_id)
    num_papers = len(paper_index)
    size_papers = getsizeof(paper_ids) + getsizeof(paper_index)
    num_citations = sum([len(s) for s in citations.values()])
    size_citations = getsizeof(citations)
    print(f'{corpus_id}\t{num_papers}\t{size_papers}\t{num_citations}\t{size_citations}')
    if size_papers + size_citations > 500_000_000:
        print('Alarm!')
        break

Corpus ID	NumPapers	SizePapers	NumCitations	SizeCitations
0	677547	41943216	659874	20971616
1	1356460	83886272	1344303	41943144
2	2007963	167772336	2017978	41943144
3	2632966	167772336	2680019	83886176
4	3250798	335544512	3350718	83886176
5	3860757	335544512	4025229	83886176
6	4443366	335544512	4684217	83886176
7	5015578	335544512	5340383	167772264
Alarm!
